In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [37]:
#寻找宝藏
import pandas as pd
import numpy as np
import time

np.random.seed(2)

N_STATES = 6   #状态数量
ACTIONS = ['left','right']
EPSILON = 0.9     #90%的纪律选择最佳动作，10%随机选择
ALPHA = 0.1       #learning rate
LAMBDA = 0.9      #Q表的消减比例
MAX_EPISODES = 13    #最大的回合数
FRESH_TIME = 0.3    

def build_q_table(n_states,actions):
    table = pd.DataFrame(np.zeros((n_states,len(actions))),
                        columns=actions)
    return table

# build_q_table(N_STATES,ACTIONS)

def choose_action(state,q_table):
    state_actions = q_table.iloc[state,:]
    if (np.random.uniform()>EPSILON) or (state_actions.all()==0):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.idxmax()
    return action_name

def get_env_feedback(S,A):
    if A == 'right':
        if S == N_STATES-2:
            S_ = 'terminal'
            R = 1
        else:
            S_ =S + 1
            R = 0
    else:
        R = 0
        if S == 0:
            S_ = S 
        else:
            S_ =S - 1
    return S_,R

def updata_env(S,episode,step_counter):    #主要是负责打印
    env_list = ['-']*(N_STATES-1)+['T']
    if S == 'terminal':
        interaction = 'Episode %s:total_steps = %s'%(episode+1,step_counter)
        print('\r{}'.format(interaction),end='')
        time.sleep(2)
        print('\r                                ',end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction),end='')
        time.sleep(FRESH_TIME)

def r1():
    q_table = build_q_table(N_STATES,ACTIONS)
    for episode in range(MAX_EPISODES):
            step_counter = 0
            S = 0
            is_terminated = False
            updata_env(S,episode,step_counter)       
            while not is_terminated:
                A = choose_action(S,q_table)
                S_,R = get_env_feedback(S,A)
#                 print(S,A)
                q_predict = q_table.loc[S,A]
                if S_ != 'terminal':
                    q_target = R+LAMBDA*q_table.iloc[S_,:].max()
                else:
                    q_target = R
                    is_terminated = True
                q_table.loc[S,A] += ALPHA*(q_target-q_predict)   #只有在第一次找到宝藏之后才会有Q值
                S = S_
                updata_env(S,episode,step_counter+1)
                step_counter += 1
    return q_table
r1()

,left,right
0,0.000001,0.005728
1,0.000271,0.032612
2,0.002454,0.111724
3,0.000073,0.343331
4,0.000810,0.745813
5,0.000000,0.000000


In [8]:
#寻找宝藏

import numpy as np
import time
import sys
import pandas as pd
if sys.version_info.major == 2:
    import Tkinter as tk
else:
    import tkinter as tk
UNIT = 40   # pixels
MAZE_H = 4  # grid height
MAZE_W = 4  # grid width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('maze')
        self.geometry('{0}x{1}'.format(MAZE_H * UNIT, MAZE_H * UNIT))
        self._build_maze()

    def _build_maze(self):
        self.canvas = tk.Canvas(self, bg='white',
                           height=MAZE_H * UNIT,
                           width=MAZE_W * UNIT)

        # create grids
        for c in range(0, MAZE_W * UNIT, UNIT):
            x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, MAZE_H * UNIT, UNIT):
            x0, y0, x1, y1 = 0, r, MAZE_W * UNIT, r
            self.canvas.create_line(x0, y0, x1, y1)

        # create origin
        origin = np.array([20, 20])

        # hell
        hell1_center = origin + np.array([UNIT * 2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0] - 15, hell1_center[1] - 15,
            hell1_center[0] + 15, hell1_center[1] + 15,
            fill='black')
        # hell
        hell2_center = origin + np.array([UNIT, UNIT * 2])
        self.hell2 = self.canvas.create_rectangle(
            hell2_center[0] - 15, hell2_center[1] - 15,
            hell2_center[0] + 15, hell2_center[1] + 15,
            fill='black')

        # create oval
        oval_center = origin + UNIT * 2
        self.oval = self.canvas.create_oval(
            oval_center[0] - 15, oval_center[1] - 15,
            oval_center[0] + 15, oval_center[1] + 15,
            fill='yellow')

        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')

        # pack all
        self.canvas.pack()

    def reset(self):
        self.update()
        time.sleep(0.5)
        self.canvas.delete(self.rect)
        origin = np.array([20, 20])
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')
        # return observation
        return self.canvas.coords(self.rect)

    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0, 0])
        if action == 0:   # up
            if s[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:   # down
            if s[1] < (MAZE_H - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:   # right
            if s[0] < (MAZE_W - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 3:   # left
            if s[0] > UNIT:
                base_action[0] -= UNIT

        self.canvas.move(self.rect, base_action[0], base_action[1])  # move agent

        s_ = self.canvas.coords(self.rect)  # next state

        # reward function
        if s_ == self.canvas.coords(self.oval):
            reward = 1
            done = True
            s_ = 'terminal'
        elif s_ in [self.canvas.coords(self.hell1), self.canvas.coords(self.hell2)]:
            reward = -1
            done = True
            s_ = 'terminal'
        else:
            reward = 0
            done = False

        return s_, reward, done

    def render(self):
        time.sleep(0.1)
        self.update()



class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        self.actions = actions  # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # choose best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose on in these actions
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if s_ != 'terminal':
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()  # next state is not terminal
        else:
            q_target = r  # next state is terminal
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)  # update

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            print(state)
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )
def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()

        while True:
            # fresh env
            env.render()

            # RL choose action based on observation
            action = RL.choose_action(str(observation))

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_))

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break

    # end of game
    print('game over')
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    RL = QLearningTable(actions=list(range(env.n_actions)))

    env.after(100, update)
    env.mainloop()

'after#6'

[5.0, 5.0, 35.0, 35.0]
[45.0, 5.0, 75.0, 35.0]
[45.0, 45.0, 75.0, 75.0]
[5.0, 45.0, 35.0, 75.0]
[5.0, 85.0, 35.0, 115.0]
terminal
[85.0, 5.0, 115.0, 35.0]
[125.0, 5.0, 155.0, 35.0]
[125.0, 45.0, 155.0, 75.0]
[125.0, 85.0, 155.0, 115.0]
[5.0, 125.0, 35.0, 155.0]
[45.0, 125.0, 75.0, 155.0]
[85.0, 125.0, 115.0, 155.0]
[125.0, 125.0, 155.0, 155.0]


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "D:\anaconda\lib\tkinter\__init__.py", line 804, in callit
    func(*args)
  File "<ipython-input-8-2301215ee5b9>", line 177, in update
    observation_, reward, done = env.step(action)
  File "<ipython-input-8-2301215ee5b9>", line 83, in step
    s = self.canvas.coords(self.rect)
  File "D:\anaconda\lib\tkinter\__init__.py", line 2761, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
